#### A Demonstration of Item-Based Collaborative filtering

*Item-based collaborative filtering is a technique of generating recommendations, where items similar to the items liked by the user is recommended to the user.*

Following are the steps:


*   Generate a user - interest matrix (For eg: user - rating matrix)
*   Here each row represents a kind of user vector, and column represents an interest/item vector.
*   Find similar items by computing similarity metric between each item vector with all other item vectors.

**Dataset used:** <br>
MovieLens Dataset 1M (http://files.grouplens.org/datasets/movielens/ml-latest-small.zip)



In [ ]:
import os
if not os.path.isfile('ml-latest-small.zip'):
  !wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
  !unzip 'ml-latest-small.zip'

In [ ]:
v1 = [1, 2 ,3]
v2 = [2, 4, 5]
v1_norm = np.linalg.norm(v1)
v2_norm = np.linalg.norm(v2)
cos_dist = np.dot(v1, v2) / (v1_norm * v2_norm)
print(cos_dist)

0.9960238411119946


In [ ]:
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
base_path_1 = 'ml-25m'
base_path_2 = 'ml-latest-small'
df_genome_scores = pd.read_csv(os.path.join(base_path_1, 'genome-scores.csv'))
df_genome_tags = pd.read_csv(os.path.join(base_path_1, 'genome-tags.csv'))
df_links = pd.read_csv(os.path.join(base_path_2, 'links.csv'))
df_movies = pd.read_csv(os.path.join(base_path_2, 'movies.csv'))
df_ratings = pd.read_csv(os.path.join(base_path_2, 'ratings.csv'))
df_tags = pd.read_csv(os.path.join(base_path_2, 'tags.csv'))

In [ ]:
df_movies.shape

(9742, 3)

In [ ]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
### CHECK GENRES ####
genres = []
def num_genres(x):
  return len(x.split('|'))

for genre in df_movies['genres']:
  for x in genre.split('|'):
    genres.append(x)
genres = set(genres)

df_movies['num_genres'] = df_movies['genres'].map(lambda x: num_genres(x))

In [ ]:
df_genome_scores

,movieId,tagId,relevance
0,1,1,0.02875
1,1,2,0.02375
2,1,3,0.06250
3,1,4,0.07575
4,1,5,0.14075
...,...,...,...
15584443,206499,1124,0.11000
15584444,206499,1125,0.04850
15584445,206499,1126,0.01325
15584446,206499,1127,0.14025


In [ ]:
df_user_ratings = pd.merge(df_ratings, df_movies, on = ['movieId'])
df_user_ratings

,userId,movieId,rating,timestamp,title,genres,num_genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5
...,...,...,...,...,...,...,...
100831,610,160341,2.5,1479545749,Bloodmoon (1997),Action|Thriller,2
100832,610,160527,4.5,1479544998,Sympathy for the Underdog (1971),Action|Crime|Drama,3
100833,610,160836,3.0,1493844794,Hazard (2005),Action|Drama|Thriller,3
100834,610,163937,3.5,1493848789,Blair Witch (2016),Horror|Thriller,2


In [ ]:
def ret_count(x):
  return len(x)

In [ ]:
df_aggr = df_user_ratings.groupby(['userId']).aggregate({'movieId':ret_count})

In [ ]:
df_aggr

,movieId
userId,
1,232
2,29
3,39
4,216
5,44
...,...
606,1115
607,187
608,831


In [ ]:
total_movies = df_movies.shape[0]
df_aggr['percentage_movies_rated'] = df_aggr['movieId'] * 100/ total_movies 

In [ ]:
df_aggr

,movieId,percentage_movies_rated
userId,,
1,232,2.381441
2,29,0.297680
3,39,0.400328
4,216,2.217204
5,44,0.451653
...,...,...
606,1115,11.445288
607,187,1.919524
608,831,8.530076


In [ ]:
sort_by_num_reviews = df_aggr.sort_values(by = 'movieId', ascending = False)
sort_by_num_reviews

,movieId,percentage_movies_rated
userId,,
414,2698,27.694519
599,2478,25.436255
474,2108,21.638267
448,1864,19.133648
274,1346,13.816465
...,...,...
442,20,0.205297
569,20,0.205297
320,20,0.205297


In [ ]:
import plotly.graph_objects as go
import plotly.express as px
val_cnts = sort_by_num_reviews['movieId'].value_counts()
indices = sort_by_num_reviews[sort_by_num_reviews['movieId'] >= 100].index
user_rating_slice = df_user_ratings.loc[df_user_ratings['userId'].isin(indices), :]

In [ ]:
cumsum = np.cumsum(sort_by_num_reviews['movieId'])
sort_by_num_reviews['data_covered'] = cumsum
sort_by_num_reviews['percent_data_covered'] = cumsum * 100 / df_user_ratings.shape[0]

In [ ]:
sort_by_num_reviews

,movieId,percentage_movies_rated,data_covered,percent_data_covered
userId,,,,
414,2698,27.694519,2698,2.675632
599,2478,25.436255,5176,5.133087
474,2108,21.638267,7284,7.223611
448,1864,19.133648,9148,9.072157
274,1346,13.816465,10494,10.406998
...,...,...,...,...
442,20,0.205297,100756,99.920663
569,20,0.205297,100776,99.940497
320,20,0.205297,100796,99.960332


In [ ]:
sort_by_num_reviews[sort_by_num_reviews['percent_data_covered'] <= 40].iloc[-1]['data_covered']

40054.0

In [ ]:
indices = sort_by_num_reviews[sort_by_num_reviews['percent_data_covered'] <= 40].index

In [ ]:
df_user_ratings_train = df_user_ratings[df_user_ratings['userId'].isin(indices)]

In [ ]:
df_user_ratings_test = df_user_ratings[~df_user_ratings['userId'].isin(indices)]

assert (df_user_ratings_train.shape[0] + df_user_ratings_test.shape[0] == df_user_ratings.shape[0])

In [ ]:
df_user_ratings.reset_index(drop=True)

,userId,movieId,rating,timestamp,title,genres,num_genres
0,18,1,3.5,1455209816,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5
1,19,1,4.0,965705637,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5
2,64,1,4.0,1161520134,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5
3,68,1,2.5,1158531426,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5
4,89,1,3.0,1520408314,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5
...,...,...,...,...,...,...,...
40049,610,160341,2.5,1479545749,Bloodmoon (1997),Action|Thriller,2
40050,610,160527,4.5,1479544998,Sympathy for the Underdog (1971),Action|Crime|Drama,3
40051,610,160836,3.0,1493844794,Hazard (2005),Action|Drama|Thriller,3
40052,610,163937,3.5,1493848789,Blair Witch (2016),Horror|Thriller,2


In [ ]:
def get_release_year(x):
  yr = x.split()[-1]
  yr = yr[1:-1]
  return yr

release_year = df_user_ratings['title'].map(lambda x: get_release_year(x))

In [ ]:
df_user_ratings = pd.merge(df_user_ratings, df_genome_scores, on = ['movieId'])

In [ ]:
df_user_table = pd.pivot_table(df_user_ratings, values = 'rating', index = ['userId'], columns = ['movieId'], fill_value = 0.0)

In [ ]:
df_user_table

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,41,42,43,44,...,176751,176935,177593,177615,177763,177765,178061,178323,178613,179119,179133,179135,179401,179817,179819,179953,180031,180045,180095,180265,180297,180497,180985,181315,182715,182823,183197,183227,184015,184253,184257,184471,185029,185435,185585,187593,187595,188301,188675,188833
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
18,3.5,3.0,0.0,0.0,0.0,4.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,4.5,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,4.0,2.5,4.0,0,0.0,0.0,0,0.0,0.0,...,0,0,4.5,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,3,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0,0,0.0,0,0.0,0.0
19,4.0,3.0,3.0,0.0,0.0,0.0,2.0,0,0.0,2.0,0.0,1.0,3,0,2.0,0.0,0.0,0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,4.0,4.0,0.0,0,0.0,0.0,0,0.0,3.0,...,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0,0,0.0,0,0.0,0.0
28,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,2.5,0.0,0,0.0,0.0,3.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0,2.5,3.5,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0,0,0.0,0,0.0,0.0
64,4.0,0.0,3.5,0.0,0.0,4.5,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,5.0,0.0,0,3.5,0.0,0.0,3.5,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0,0.0,4.0,4.5,4.0,0,4.0,0.0,0,0.0,0.0,...,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0,0,0.0,0,0.0,0.0
68,2.5,2.5,2.0,0.0,2.0,4.0,2.0,0,0.0,4.5,4.5,0.0,0,0,0.0,3.5,3.5,2,1.5,0.0,0.0,0.0,0.0,0.0,3.5,3.0,0.0,0.0,0.0,0,3.5,0.0,1.0,0.0,0,4.0,0.0,0,0.0,3.0,...,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0,0,0.0,0,0.0,0.0
89,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,2.5,0.0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,0,0,0.0,0.0,0.0,2,0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0,0,0.0,0,0.0,0.0
91,4.0,3.0,3.0,0.0,0.0,5.0,0.0,0,0.0,3.5,0.0,0.0,0,0,0.0,4.5,0.0,0,2.0,0.0,4.0,3.5,0.0,0.0,2.0,0.0,0.0,0.0,4.0,0,0.0,4.0,0.0,3.0,0,1.5,4.0,0,0.0,0.0,...,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0,0,0.0,0,0.0,0.0
105,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,4.5,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,3.5,0.0,0.0,0,0.0,0.0,0,0.0,0.0,...,0,0,0.0,0.0,0.0,0,0,0.0,4,0,5,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0,0,0.0,0,0.0,0.0
111,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,4.5,0.0,0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,2.5,0.0,0,4.0,0.0,0,0.0,0.0,...,0,0,0.0,0.0,0.0,0,0,3.5,0,0,0,0,0,0,0.0,3.5,0.0,0,0,4.5,0,0,0.0,0.0,0.0,0.0,3.5,3.5,3.5,0,4,0,0.0,0,0.0,0,0.0,0,0.0,0.0


In [ ]:
from scipy.stats import pearsonr
def compute_correlation(x, y):
  return pearsonr(x, y)

In [ ]:
from sklearn.metrics import pairwise_distances
user_similarity = pairwise_distances(df_user_table, metric = 'correlation')
user_similarity = pd.DataFrame(user_similarity)
user_similarity.index = df_user_table.index
user_similarity.columns = df_user_table.index

In [ ]:
user_similarity

userId,18,19,28,64,68,89,91,105,111,140,177,182,217,219,226,232,249,274,288,298,305,307,318,380,387,414,438,448,474,477,480,483,489,534,555,561,590,599,600,603,606,608,610
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
18,0.000000,0.806386,6.513040e-01,6.749562e-01,0.596642,9.146078e-01,0.675384,0.641927,7.751630e-01,0.764935,0.659819,0.721155,0.823052,0.603782,6.972968e-01,6.579647e-01,5.471153e-01,0.629133,0.720768,5.858044e-01,0.556268,6.638430e-01,0.651426,5.936405e-01,7.298183e-01,6.320995e-01,0.704546,6.415292e-01,7.474009e-01,0.655100,6.333906e-01,6.125439e-01,0.683699,6.701890e-01,8.237496e-01,0.579865,6.060339e-01,0.639518,0.744790,8.264790e-01,7.476073e-01,0.663611,0.622750
19,0.806386,0.000000,8.349492e-01,7.054169e-01,0.690138,9.217087e-01,0.617170,0.930309,8.910134e-01,0.816770,0.772860,0.719016,0.564098,0.711367,7.012700e-01,8.597840e-01,8.116399e-01,0.686761,0.627351,7.974774e-01,0.825091,7.159955e-01,0.875236,6.875350e-01,7.287037e-01,6.591052e-01,0.750672,7.022997e-01,7.120003e-01,0.711062,7.099942e-01,7.007898e-01,0.709815,8.189081e-01,5.835851e-01,0.700480,6.725222e-01,0.651743,0.664079,7.266028e-01,8.003193e-01,0.642284,0.857695
28,0.651304,0.834949,1.110223e-16,6.988138e-01,0.664209,9.658152e-01,0.700200,0.717131,8.948519e-01,0.669273,0.784839,0.691727,0.829534,0.606181,7.398802e-01,6.640173e-01,6.920232e-01,0.657941,0.703428,6.827998e-01,0.627984,6.985490e-01,0.726496,6.828187e-01,7.471529e-01,6.168909e-01,0.698167,7.083042e-01,7.130804e-01,0.714552,6.043718e-01,6.865147e-01,0.748370,8.195605e-01,8.030587e-01,0.658958,6.373095e-01,0.671576,0.791888,8.093143e-01,7.559143e-01,0.654726,0.684827
64,0.674956,0.705417,6.988138e-01,2.220446e-16,0.564790,9.002952e-01,0.592590,0.750280,8.012197e-01,0.712784,0.678541,0.604462,0.757634,0.599396,5.547055e-01,7.034499e-01,7.200329e-01,0.591226,0.598960,6.736040e-01,0.710528,5.717108e-01,0.767213,6.475527e-01,6.463684e-01,5.832746e-01,0.643615,6.704841e-01,6.251040e-01,0.645987,4.825359e-01,6.251277e-01,0.612279,7.706448e-01,7.081044e-01,0.637067,5.981764e-01,0.612501,0.606817,7.343297e-01,6.964497e-01,0.511817,0.716922
68,0.596642,0.690138,6.642091e-01,5.647896e-01,0.000000,8.606034e-01,0.653721,0.667361,6.659281e-01,0.671160,0.512751,0.664503,0.731535,0.525434,5.662334e-01,5.260990e-01,5.078941e-01,0.496382,0.594781,5.524322e-01,0.617590,5.593062e-01,0.666000,5.213598e-01,7.074422e-01,4.759290e-01,0.553375,5.535679e-01,6.516271e-01,0.614002,5.138067e-01,5.586472e-01,0.558701,6.121266e-01,7.332397e-01,0.540166,5.437916e-01,0.526204,0.627131,7.931013e-01,7.000115e-01,0.507170,0.597858
89,0.914608,0.921709,9.658152e-01,9.002952e-01,0.860603,1.110223e-16,0.927803,0.907197,8.000199e-01,0.953969,0.835215,0.924466,0.948316,0.901227,8.829857e-01,8.513181e-01,8.355383e-01,0.875824,0.898771,8.471830e-01,0.914126,8.926139e-01,0.897937,8.891138e-01,9.206353e-01,8.651527e-01,0.895223,8.538284e-01,8.958337e-01,0.905094,8.883343e-01,8.634617e-01,0.901400,8.504046e-01,9.449104e-01,0.903457,8.958349e-01,0.902881,0.906945,9.463307e-01,9.196362e-01,0.912516,0.885568
91,0.675384,0.617170,7.002000e-01,5.925902e-01,0.653721,9.278034e-01,0.000000,0.813093,8.944552e-01,0.692415,0.735706,0.613737,0.683448,0.596286,6.491307e-01,7.776293e-01,7.363840e-01,0.589053,0.590509,7.003810e-01,0.702065,6.389563e-01,0.800146,6.040532e-01,6.268812e-01,6.175802e-01,0.687540,6.663268e-01,6.794958e-01,0.648242,5.778927e-01,6.487524e-01,0.648921,7.813490e-01,6.735814e-01,0.618265,5.747763e-01,0.595888,0.649597,7.122723e-01,7.482651e-01,0.559009,0.726638
105,0.641927,0.930309,7.171308e-01,7.502803e-01,0.667361,9.071971e-01,0.813093,0.000000,7.906903e-01,0.816307,0.686149,0.766701,0.952177,0.709103,7.835686e-01,6.910932e-01,6.254403e-01,0.694297,0.800948,6.280008e-01,0.617876,7.500138e-01,0.695731,7.101715e-01,7.817503e-01,6.964342e-01,0.778468,7.483232e-01,7.913162e-01,0.755337,7.084773e-01,6.690005e-01,0.739347,7.800233e-01,9.234658e-01,0.734276,7.202826e-01,0.744631,0.807464,8.673204e-01,7.389422e-01,0.7

In [ ]:
movie_similarity = pairwise_distances(df_user_table.T, metric = 'correlation')

In [ ]:
movie_similarity = pd.DataFrame(movie_similarity)
movie_similarity.index = df_user_table.columns.values
movie_similarity.columns = df_user_table.columns.values

In [ ]:
def get_similar_users(user, k = 5):
  return user_similarity[user].sort_values()[-k:].index.values

def get_movies_watched_by_user(user):
  movies_watched_by_user = df_user_ratings.loc[df_user_ratings['userId'] == user, 'movieId']
  return movies_watched_by_user.values

def get_ratings_given_by_user(user):
  movies_rated_by_user = df_user_ratings.loc[df_user_ratings['userId'] == user, ['movieId','rating']]
  return movies_rated_by_user


get_ratings_given_by_user(91)

,movieId,rating
32,1,4.0
226,3,3.0
284,6,5.0
400,47,4.5
607,50,4.5
...,...,...
87551,8501,4.0
87555,8507,2.5
87564,8581,4.0
87568,27317,4.5


In [ ]:
movies_watched = get_movies_watched_by_user(91)
similar_movies = movie_similarity[movies_watched]
similar_movies

,1,3,6,47,50,110,151,163,223,231,235,260,296,316,349,356,367,457,480,500,543,552,553,590,592,593,608,648,733,736,780,919,923,1073,1080,1089,1092,1097,1127,1136,...,4987,5165,5294,5346,5438,5479,5588,5640,5650,5749,6203,6219,6283,6327,6375,6395,6461,6620,6629,6638,6678,6731,6744,6820,6963,7118,7308,7492,7561,7650,7802,8016,8147,8266,8426,8501,8507,8581,27317,27873
1,0.000000,0.361097,0.335116,0.158582,0.238453,0.214072,0.598937,0.421685,0.231384,0.214509,0.444600,0.091282,0.164038,0.308674,0.453022,0.142121,0.124870,0.146102,0.100647,0.179798,0.417947,0.431507,0.441899,0.368657,0.188997,0.189098,0.171945,0.176853,0.239423,0.212061,0.139574,0.202814,0.315196,0.229591,0.335363,0.217045,0.429654,0.143217,0.351925,0.182577,...,0.772603,0.753467,0.608086,0.773400,0.639938,0.673643,0.753467,0.825675,0.639375,0.825675,0.784283,0.735802,0.503147,0.753467,0.674565,0.825675,0.813959,0.477594,0.742870,0.728693,0.825675,0.628226,0.727380,0.581806,0.753892,0.885206,0.825675,0.825675,0.825675,0.848643,0.546523,0.755370,0.825675,0.733920,0.825675,0.740205,0.656837,0.643207,0.681684,0.701350
2,0.197686,0.520185,0.438007,0.132538,0.338576,0.344115,0.566255,0.466045,0.276409,0.281577,0.491596,0.181831,0.213472,0.338891,0.465955,0.179339,0.226049,0.239056,0.147400,0.135430,0.568508,0.460290,0.506106,0.443040,0.303543,0.245820,0.234593,0.143019,0.225467,0.211982,0.128563,0.240110,0.395878,0.214907,0.355241,0.319987,0.614559,0.216913,0.433959,0.228023,...,0.806824,0.744754,0.717425,0.783439,0.640928,0.661147,0.744754,0.833398,0.689588,0.833398,0.784023,0.663342,0.621559,0.764389,0.683328,0.833398,0.773710,0.589304,0.800077,0.731518,0.833398,0.699112,0.748142,0.672393,0.764795,0.890292,0.833398,0.833398,0.833398,0.855349,0.593268,0.722372,0.833398,0.734074,0.833398,0.775497,0.614061,0.806385,0.778905,0.714581
3,0.361097,0.000000,0.673720,0.462628,0.505334,0.471818,0.580927,0.560809,0.509937,0.287829,0.571195,0.380054,0.439196,0.443470,0.509964,0.412685,0.377719,0.405253,0.416385,0.411419,0.312530,0.604348,0.318948,0.460966,0.453647,0.435494,0.403234,0.510847,0.434559,0.449321,0.461447,0.431763,0.469196,0.440355,0.571478,0.451166,0.311613,0.304746,0.427504,0.388654,...,0.671123,0.830294,0.633239,0.658321,0.482737,0.622804,0.830294,0.760000,0.460771,0.760000,0.745442,0.861436,0.711210,0.830294,0.592693,0.760000,0.869607,0.546869,0.616000,0.623833,0.760000,0.854750,0.737272,0.692127,0.840553,0.841959,0.760000,0.760000,0.760000,0.791622,0.795212,0.852654,0.760000,0.658419,0.760000,0.868444,0.923491,0.880466,0.827579,0.689826
4,0.891047,1.000000,1.000000,0.841014,1.000000,0.910780,0.716842,1.000000,0.876496,0.795353,1.000000,0.855827,0.839260,1.000000,0.828659,0.848435,0.830902,0.858618,0.872943,0.799119,0.820971,0.697646,1.000000,1.000000,0.877169,0.842317,0.834691,0.848089,1.000000,0.830872,0.904681,0.857466,0.794262,0.756676,1.000000,1.000000,1.000000,0.908879,0.770214,0.810010,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.774187,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.553696,1.000000,1.000000,1.000000
5,0.571606,0.652703,0.762871,0.579723,0.634723,0.576711,0.619519,0.747940,0.728858,0.613811,0.772505,0.571937,0.560271,0.657420,0.474007,0.480668,0.591007,0.542595,0.529842,0.511020,0.706884,0.410677,0.654016,0.651241,0.509422,0.581943,0.554037,0.580760,0.583307,0.611913,0.583243,0.532974,0.488032,0.512583,0.619424,0.679323,0.706786,0.536610,0.643732,0.553567,...,0.921554,1.000000,0.850029,0.907869,0.732672,0.759193,1.000000,1.000000,0.864059,1.000000,1.000000,1.000000,0.824098,0.868441,0.747399,1.000000,1.000000,0.698907,0.851158,0.868570,1.000000,1.000000,0.903014,0.824139,0.860943,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.755232,1.000000,0.826767,1.000000,1.000000,0.766709,0.845559,1.000000,0.725996
...,

#### MAKING SOME RECOMMENDATIONS

In [ ]:
def get_similar_movies(movieId):
  movie_sims = movie_similarity.T[movieId].sort_values()[:5].index
  movie_titles = df_movies[df_movies['movieId'].isin(movie_sims)]['title']
  return movie_titles

print("Test Movie Name: ", df_movies[df_movies['movieId'] == 50][['title', 'genres']].values[0][0])
print("Top 5 similar movies: \n")
movie_sims = get_similar_movies(50)
for movie in movie_sims:
  print(df_movies.loc[df_movies['title'] == movie, ['title', 'genres']].values[0][0])
# def get_recommendations(user_id):
#   user_vector = df_user_table[user_id]
#   for movies in user_vector:
#     if user_vector[movies] >= 4.0:


Test Movie Name:  Usual Suspects, The (1995)
Top 5 similar movies: 

Usual Suspects, The (1995)
Godfather, The (1972)
Reservoir Dogs (1992)
Goodfellas (1990)
Godfather: Part II, The (1974)
